In [ ]:
### Python-dotenv



In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')


'your api key value'

In [ ]:
# Chat Models: GPT-3.5 Turbo and GPT-4

In [3]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

llm = ChatOpenAI()
output = llm.invoke('Explain quantum mechanics in one sentence.')
print(output.content)


Quantum mechanics is the branch of physics that studies the behavior of particles at the smallest scales, where classical physics principles do not apply and phenomena such as superposition and entanglement are observed.


In [4]:
help(ChatOpenAI)


Help on class ChatOpenAI in module langchain_openai.chat_models.base:

class ChatOpenAI(langchain_core.language_models.chat_models.BaseChatModel)
 |  ChatOpenAI(*, name: Optional[str] = None, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, client: Any = None, async_client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, api_key: Optional[pydantic.v1.types.SecretStr] = None, base_url: Optional[str] = None, organization: Optional[str] = None, openai_proxy: Optional[str] = None, timeout: Union[float, Tuple[float, float], Any, NoneType] = None, max_retries: int = 2, streaming: bool = False, n: int = 1, max_tokens: Optional[int] =

In [5]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

message = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(message)
print(output.content)


Quantenmechanik ist ein physikalisches Modell, das das Verhalten von Teilchen auf atomarer und subatomarer Ebene beschreibt.


### In-Memory Cache - Caching LLM Responses

In [7]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')


UsageError: Line magic function `%%time` not found.


In [8]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 177 ms, sys: 76.4 ms, total: 253 ms
Wall time: 4.06 s


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

In [9]:
%%time
llm.invoke(prompt)


CPU times: user 677 µs, sys: 1.78 ms, total: 2.45 ms
Wall time: 2.46 ms


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

### SQLite Caching

In [10]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.sqlite.db"))

prompt2 = "Tell me a joke"


UsageError: Line magic function `%%time` not found.


In [12]:
%%time
# first request (not in cache, takes longer)

llm.invoke(prompt2)



CPU times: user 22.1 ms, sys: 30.9 ms, total: 53 ms
Wall time: 1.46 s


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired."

In [13]:
%%time
# second request (cached, faster)
llm.invoke(prompt2)

CPU times: user 13.1 ms, sys: 20.5 ms, total: 33.6 ms
Wall time: 46.1 ms


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired."

## LLM Streaming


In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

(Verse 1)
In the dead of night, the moon hangs high
The raven calls, a haunting cry
It's a dance of darkness, a symphony of light
As they circle each other in the endless night

(Chorus)
Moon and Raven, a twisted pair
In the shadows, they both stare
One full of mystery, the other full of dread
Together they roam, in the sky overhead

(Verse 2)
The raven whispers secrets to the moon
Of lost souls and forgotten tombs
The moon listens, silent and still
As the raven's caw sends a chill

(Chorus)
Moon and Raven, a twisted pair
In the shadows, they both stare
One full of mystery, the other full of dread
Together they roam, in the sky overhead

(Bridge)
They're a reminder of the balance we seek
Between light and dark, strong and meek
They're a symbol of the eternal fight
Between day and night, wrong and right

(Chorus)
Moon and Raven, a twisted pair
In the shadows, they both stare
One full of mystery, the other full of dread
Together they roam, in the sky overhead

(Outro)
So when you see the

In [15]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

(Verse 1)
In the dead of night, under the silver moon
A raven calls out, a haunting tune
Its wings spread wide, black as the night
A creature of darkness, out of sight

(Chorus)
Moon and raven, dancing in the sky
A mysterious pair, never asking why
Their spirits soar, in the dead of night
A symphony of darkness, a beautiful sight

(Verse 2)
The moon shines bright, casting shadows below
The raven calls out, a chilling echo
They dance together, in perfect harmony
A bond unbreakable, for all eternity

(Chorus)
Moon and raven, dancing in the sky
A mysterious pair, never asking why
Their spirits soar, in the dead of night
A symphony of darkness, a beautiful sight

(Bridge)
The moonlight glistens, on the raven's wings
As it soars through the night, a creature of kings
The moon watches on, a silent sentinel
As the raven sings, a haunting melody

(Chorus)
Moon and raven, dancing in the sky
A mysterious pair, never asking why
Their spirits soar, in the dead of night
A symphony of darkness, a be